# PROBLEM STATEMENT:

The management of a bakery store reached out to you, a data scientist, with the aim of knowing the best pasteries to sell together for an upcoming combo deal period in the bakery. The management wants to know the pasteries commonly bought together in their bakery, to enable them come up with 'eye catching' combo deals for their customers. 




# SOLUTION:

Exploratory data analysis will be carried out to understand overall sales performance.

Pasteries that are most frequently bought together will be identified with the aid of Apriori algorithm to aid development of combo deals for cross-selling 

# IMPORT LIBRARIES

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules



# READ FILE INTO PANDAS DATAFRAME

In [2]:
Bakery = pd.read_csv('Bakery sales.csv')
Bakery

,Unnamed: 0,date,time,ticket_number,article,Quantity,unit_price
0,0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,4,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,5,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,8,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"
...,...,...,...,...,...,...,...
234000,511387,2022-09-30,18:52,288911.0,COUPE,1.0,"0,15 €"
234001,511388,2022-09-30,18:52,288911.0,BOULE 200G,1.0,"1,20 €"
234002,511389,2022-09-30,18:52,288911.0,COUPE,2.0,"0,15 €"
234003,511392,2022-09-30,18:55,288912.0,TRADITIONAL BAGUETTE,1.0,"1,30 €"


# DATA CLEANING AND FEATURE ENGINEERING 

# Drop first  column

In [3]:
Bakery.drop(Bakery.columns[[0]], axis = 1, inplace = True)

In [4]:
Bakery

,date,time,ticket_number,article,Quantity,unit_price
0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €"
1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €"
2,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €"
3,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €"
4,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €"
...,...,...,...,...,...,...
234000,2022-09-30,18:52,288911.0,COUPE,1.0,"0,15 €"
234001,2022-09-30,18:52,288911.0,BOULE 200G,1.0,"1,20 €"
234002,2022-09-30,18:52,288911.0,COUPE,2.0,"0,15 €"
234003,2022-09-30,18:55,288912.0,TRADITIONAL BAGUETTE,1.0,"1,30 €"


# Overview of data

In [5]:
Bakery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           234005 non-null  object 
 1   time           234005 non-null  object 
 2   ticket_number  234005 non-null  float64
 3   article        234005 non-null  object 
 4   Quantity       234005 non-null  float64
 5   unit_price     234005 non-null  object 
dtypes: float64(2), object(4)
memory usage: 10.7+ MB


# Check missing values

In [6]:
Bakery.isnull().sum()

date             0
time             0
ticket_number    0
article          0
Quantity         0
unit_price       0
dtype: int64

# Merge 'date' and 'time' columns 

In [7]:
Bakery['Date'] = Bakery['date'] + ' ' + Bakery['time']
Bakery

,date,time,ticket_number,article,Quantity,unit_price,Date
0,2021-01-02,08:38,150040.0,BAGUETTE,1.0,"0,90 €",2021-01-02 08:38
1,2021-01-02,08:38,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €",2021-01-02 08:38
2,2021-01-02,09:14,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €",2021-01-02 09:14
3,2021-01-02,09:14,150041.0,PAIN,1.0,"1,15 €",2021-01-02 09:14
4,2021-01-02,09:25,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €",2021-01-02 09:25
...,...,...,...,...,...,...,...
234000,2022-09-30,18:52,288911.0,COUPE,1.0,"0,15 €",2022-09-30 18:52
234001,2022-09-30,18:52,288911.0,BOULE 200G,1.0,"1,20 €",2022-09-30 18:52
234002,2022-09-30,18:52,288911.0,COUPE,2.0,"0,15 €",2022-09-30 18:52
234003,2022-09-30,18:55,288912.0,TRADITIONAL BAGUETTE,1.0,"1,30 €",2022-09-30 18:55


# Drop 'date' and 'time' columns

In [8]:
Bakery.drop(['date'], axis = 1, inplace = True)
Bakery.drop(['time'], axis = 1, inplace = True)

In [9]:
Bakery

,ticket_number,article,Quantity,unit_price,Date
0,150040.0,BAGUETTE,1.0,"0,90 €",2021-01-02 08:38
1,150040.0,PAIN AU CHOCOLAT,3.0,"1,20 €",2021-01-02 08:38
2,150041.0,PAIN AU CHOCOLAT,2.0,"1,20 €",2021-01-02 09:14
3,150041.0,PAIN,1.0,"1,15 €",2021-01-02 09:14
4,150042.0,TRADITIONAL BAGUETTE,5.0,"1,20 €",2021-01-02 09:25
...,...,...,...,...,...
234000,288911.0,COUPE,1.0,"0,15 €",2022-09-30 18:52
234001,288911.0,BOULE 200G,1.0,"1,20 €",2022-09-30 18:52
234002,288911.0,COUPE,2.0,"0,15 €",2022-09-30 18:52
234003,288912.0,TRADITIONAL BAGUETTE,1.0,"1,30 €",2022-09-30 18:55


# Strip non-numerical entity from 'unit_price' column

In [10]:
Bakery['unit_price'] = Bakery['unit_price'].str.replace(',' , '.')

In [11]:
Bakery

,ticket_number,article,Quantity,unit_price,Date
0,150040.0,BAGUETTE,1.0,0.90 €,2021-01-02 08:38
1,150040.0,PAIN AU CHOCOLAT,3.0,1.20 €,2021-01-02 08:38
2,150041.0,PAIN AU CHOCOLAT,2.0,1.20 €,2021-01-02 09:14
3,150041.0,PAIN,1.0,1.15 €,2021-01-02 09:14
4,150042.0,TRADITIONAL BAGUETTE,5.0,1.20 €,2021-01-02 09:25
...,...,...,...,...,...
234000,288911.0,COUPE,1.0,0.15 €,2022-09-30 18:52
234001,288911.0,BOULE 200G,1.0,1.20 €,2022-09-30 18:52
234002,288911.0,COUPE,2.0,0.15 €,2022-09-30 18:52
234003,288912.0,TRADITIONAL BAGUETTE,1.0,1.30 €,2022-09-30 18:55


In [12]:
Bakery['unit_price'] = Bakery['unit_price'].str.replace(r'[^\d\.]+' , '', regex= True)

In [13]:
Bakery

,ticket_number,article,Quantity,unit_price,Date
0,150040.0,BAGUETTE,1.0,0.90,2021-01-02 08:38
1,150040.0,PAIN AU CHOCOLAT,3.0,1.20,2021-01-02 08:38
2,150041.0,PAIN AU CHOCOLAT,2.0,1.20,2021-01-02 09:14
3,150041.0,PAIN,1.0,1.15,2021-01-02 09:14
4,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,2021-01-02 09:25
...,...,...,...,...,...
234000,288911.0,COUPE,1.0,0.15,2022-09-30 18:52
234001,288911.0,BOULE 200G,1.0,1.20,2022-09-30 18:52
234002,288911.0,COUPE,2.0,0.15,2022-09-30 18:52
234003,288912.0,TRADITIONAL BAGUETTE,1.0,1.30,2022-09-30 18:55


# Convert 'unit price' to float datatype

In [14]:
Bakery['unit_price'] = Bakery['unit_price'].astype('float')

# Create total price column

In [15]:
Bakery['Total_Price'] = Bakery['unit_price'] * Bakery['Quantity']

In [16]:
Bakery

,ticket_number,article,Quantity,unit_price,Date,Total_Price
0,150040.0,BAGUETTE,1.0,0.90,2021-01-02 08:38,0.90
1,150040.0,PAIN AU CHOCOLAT,3.0,1.20,2021-01-02 08:38,3.60
2,150041.0,PAIN AU CHOCOLAT,2.0,1.20,2021-01-02 09:14,2.40
3,150041.0,PAIN,1.0,1.15,2021-01-02 09:14,1.15
4,150042.0,TRADITIONAL BAGUETTE,5.0,1.20,2021-01-02 09:25,6.00
...,...,...,...,...,...,...
234000,288911.0,COUPE,1.0,0.15,2022-09-30 18:52,0.15
234001,288911.0,BOULE 200G,1.0,1.20,2022-09-30 18:52,1.20
234002,288911.0,COUPE,2.0,0.15,2022-09-30 18:52,0.30
234003,288912.0,TRADITIONAL BAGUETTE,1.0,1.30,2022-09-30 18:55,1.30


# Re-organize columns

In [17]:
new_col = ['Date', 'ticket_number', 'article', 'unit_price', 'Quantity', 'Total_Price']
Bakery = Bakery[new_col]
Bakery

,Date,ticket_number,article,unit_price,Quantity,Total_Price
0,2021-01-02 08:38,150040.0,BAGUETTE,0.90,1.0,0.90
1,2021-01-02 08:38,150040.0,PAIN AU CHOCOLAT,1.20,3.0,3.60
2,2021-01-02 09:14,150041.0,PAIN AU CHOCOLAT,1.20,2.0,2.40
3,2021-01-02 09:14,150041.0,PAIN,1.15,1.0,1.15
4,2021-01-02 09:25,150042.0,TRADITIONAL BAGUETTE,1.20,5.0,6.00
...,...,...,...,...,...,...
234000,2022-09-30 18:52,288911.0,COUPE,0.15,1.0,0.15
234001,2022-09-30 18:52,288911.0,BOULE 200G,1.20,1.0,1.20
234002,2022-09-30 18:52,288911.0,COUPE,0.15,2.0,0.30
234003,2022-09-30 18:55,288912.0,TRADITIONAL BAGUETTE,1.30,1.0,1.30


# Rename columns

In [18]:
Bakery.rename(columns = {'ticket_number' : 'Ticket_Number', 'article' : 'Article', 'unit_price' : 'Unit_Price' }, inplace = True)
Bakery

,Date,Ticket_Number,Article,Unit_Price,Quantity,Total_Price
0,2021-01-02 08:38,150040.0,BAGUETTE,0.90,1.0,0.90
1,2021-01-02 08:38,150040.0,PAIN AU CHOCOLAT,1.20,3.0,3.60
2,2021-01-02 09:14,150041.0,PAIN AU CHOCOLAT,1.20,2.0,2.40
3,2021-01-02 09:14,150041.0,PAIN,1.15,1.0,1.15
4,2021-01-02 09:25,150042.0,TRADITIONAL BAGUETTE,1.20,5.0,6.00
...,...,...,...,...,...,...
234000,2022-09-30 18:52,288911.0,COUPE,0.15,1.0,0.15
234001,2022-09-30 18:52,288911.0,BOULE 200G,1.20,1.0,1.20
234002,2022-09-30 18:52,288911.0,COUPE,0.15,2.0,0.30
234003,2022-09-30 18:55,288912.0,TRADITIONAL BAGUETTE,1.30,1.0,1.30


# Convert Date to datetime 

In [19]:
Bakery['Date'] = pd.to_datetime(Bakery['Date'], infer_datetime_format = True, errors = 'coerce')

# Extract components from 'Date' column

In [20]:
#Extract hour of the day
Bakery['Hour'] = Bakery['Date'].dt.hour
def hour_type(hour):
    if hour < 12:
        h = 'Morning'
    if hour >= 12:
        h = 'Mid-day'
    if  hour >= 16:
        h = 'Evening'
    return h
Bakery['Hour_Type'] = Bakery['Hour'].apply(hour_type)

#Extract Day
Bakery['Day_Of_Month'] = Bakery['Date'].dt.day_name()

#Weekday or Weekend
Bakery['Day_Type'] = Bakery['Date'].dt.dayofweek
Bakery['Day_Type'] = Bakery['Day_Type'].apply(lambda day: 'Weekend' if day > 4 else 'Weekday')

#Extract Date
Bakery['Date_of_Month'] = Bakery['Date'].dt.day

#Extract Month
Bakery['Month'] = Bakery['Date'].dt.month_name()

#Extract year
Bakery['Year'] = Bakery['Date'].dt.year

Bakery

,Date,Ticket_Number,Article,Unit_Price,Quantity,Total_Price,Hour,Hour_Type,Day_Of_Month,Day_Type,Date_of_Month,Month,Year
0,2021-01-02 08:38:00,150040.0,BAGUETTE,0.90,1.0,0.90,8,Morning,Saturday,Weekend,2,January,2021
1,2021-01-02 08:38:00,150040.0,PAIN AU CHOCOLAT,1.20,3.0,3.60,8,Morning,Saturday,Weekend,2,January,2021
2,2021-01-02 09:14:00,150041.0,PAIN AU CHOCOLAT,1.20,2.0,2.40,9,Morning,Saturday,Weekend,2,January,2021
3,2021-01-02 09:14:00,150041.0,PAIN,1.15,1.0,1.15,9,Morning,Saturday,Weekend,2,January,2021
4,2021-01-02 09:25:00,150042.0,TRADITIONAL BAGUETTE,1.20,5.0,6.00,9,Morning,Saturday,Weekend,2,January,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
234000,2022-09-30 18:52:00,288911.0,COUPE,0.15,1.0,0.15,18,Evening,Friday,Weekday,30,September,2022
234001,2022-09-30 18:52:00,288911.0,BOULE 200G,1.20,1.0,1.20,18,Evening,Friday,Weekday,30,September,2022
234002,2022-09-30 18:52:00,288911.0,COUPE,0.15,2.0,0.30,18,Evening,Friday,Weekday,30,September,2022
234003,2022-09-30 18:55:00,288912.0,TRADITIONAL BAGUETTE,1.30,1.0,1.30,18,Evening,Friday,Weekday,30,September,2022


In [21]:
Bakery['Hour_Type'].unique()

array(['Morning', 'Mid-day', 'Evening'], dtype=object)

# Merge Month and Year Column, 'Month_Year'  to enable analysis of year over year sales

In [22]:
Bakery['Month_Year'] = Bakery['Month'] + ' ' + Bakery['Year'].map(str)
Bakery

,Date,Ticket_Number,Article,Unit_Price,Quantity,Total_Price,Hour,Hour_Type,Day_Of_Month,Day_Type,Date_of_Month,Month,Year,Month_Year
0,2021-01-02 08:38:00,150040.0,BAGUETTE,0.90,1.0,0.90,8,Morning,Saturday,Weekend,2,January,2021,January 2021
1,2021-01-02 08:38:00,150040.0,PAIN AU CHOCOLAT,1.20,3.0,3.60,8,Morning,Saturday,Weekend,2,January,2021,January 2021
2,2021-01-02 09:14:00,150041.0,PAIN AU CHOCOLAT,1.20,2.0,2.40,9,Morning,Saturday,Weekend,2,January,2021,January 2021
3,2021-01-02 09:14:00,150041.0,PAIN,1.15,1.0,1.15,9,Morning,Saturday,Weekend,2,January,2021,January 2021
4,2021-01-02 09:25:00,150042.0,TRADITIONAL BAGUETTE,1.20,5.0,6.00,9,Morning,Saturday,Weekend,2,January,2021,January 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234000,2022-09-30 18:52:00,288911.0,COUPE,0.15,1.0,0.15,18,Evening,Friday,Weekday,30,September,2022,September 2022
234001,2022-09-30 18:52:00,288911.0,BOULE 200G,1.20,1.0,1.20,18,Evening,Friday,Weekday,30,September,2022,September 2022
234002,2022-09-30 18:52:00,288911.0,COUPE,0.15,2.0,0.30,18,Evening,Friday,Weekday,30,September,2022,September 2022
234003,2022-09-30 18:55:00,288912.0,TRADITIONAL BAGUETTE,1.30,1.0,1.30,18,Evening,Friday,Weekday,30,September,2022,September 2022


In [23]:
#Check dataset info
Bakery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234005 entries, 0 to 234004
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           234005 non-null  datetime64[ns]
 1   Ticket_Number  234005 non-null  float64       
 2   Article        234005 non-null  object        
 3   Unit_Price     234005 non-null  float64       
 4   Quantity       234005 non-null  float64       
 5   Total_Price    234005 non-null  float64       
 6   Hour           234005 non-null  int64         
 7   Hour_Type      234005 non-null  object        
 8   Day_Of_Month   234005 non-null  object        
 9   Day_Type       234005 non-null  object        
 10  Date_of_Month  234005 non-null  int64         
 11  Month          234005 non-null  object        
 12  Year           234005 non-null  int64         
 13  Month_Year     234005 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(6

# EXPLORATORY DATA ANALYSIS

# Summary Statistics

In [24]:
Bakery.describe()

,Ticket_Number,Unit_Price,Quantity,Total_Price,Hour,Date_of_Month,Year
count,234005.000000,234005.000000,234005.000000,234005.000000,234005.000000,234005.000000,234005.000000
mean,219201.258738,1.662195,1.538377,2.387303,11.072037,16.014876,2021.438388
std,40053.223896,1.722720,1.289603,2.635041,2.774393,8.763694,0.496191
min,150040.000000,0.000000,-200.000000,-200.000000,7.000000,1.000000,2021.000000
25%,184754.000000,1.100000,1.000000,1.200000,9.000000,9.000000,2021.000000
50%,218807.000000,1.200000,1.000000,1.600000,11.000000,16.000000,2021.000000
75%,253927.000000,1.500000,2.000000,2.600000,12.000000,24.000000,2022.000000
max,288913.000000,60.000000,200.000000,200.000000,20.000000,31.000000,2022.000000


# Filter Total price > 0

In [25]:
Bakery = Bakery[Bakery['Total_Price'] > 0]

# Summary statistics

In [26]:
Bakery.describe()

,Ticket_Number,Unit_Price,Quantity,Total_Price,Hour,Date_of_Month,Year
count,232679.000000,232679.000000,232679.000000,232679.000000,232679.000000,232679.000000,232679.000000
mean,219173.584346,1.659715,1.556462,2.418425,11.069796,16.017715,2021.437568
std,40067.194649,1.718190,1.198053,2.565597,2.772819,8.761820,0.496088
min,150040.000000,0.070000,1.000000,0.070000,7.000000,1.000000,2021.000000
25%,184728.000000,1.100000,1.000000,1.200000,9.000000,9.000000,2021.000000
50%,218677.000000,1.200000,1.000000,1.600000,11.000000,16.000000,2021.000000
75%,253975.000000,1.500000,2.000000,2.600000,12.000000,24.000000,2022.000000
max,288913.000000,60.000000,200.000000,200.000000,20.000000,31.000000,2022.000000


# Yearly Sales performance In Terms Of Total Units Sold

In [27]:
Yearly_Sales = Bakery.groupby('Year')['Quantity'].sum().reset_index()

Yearly_Sales

,Year,Quantity
0,2021,204329.0
1,2022,157827.0


# Year over Year Sales Performance In Terms Of Units Sold

In [28]:
Month_Yearly_Sales = Bakery.groupby('Month_Year')['Quantity'].sum().reset_index().sort_values(by = 'Month_Year',ascending = False)
Month_Yearly_Sales

,Month_Year,Quantity
20,September 2022,13463.0
19,September 2021,15296.0
18,October 2021,15044.0
17,November 2021,11868.0
16,May 2022,17908.0
15,May 2021,19540.0
14,March 2022,11891.0
13,March 2021,13895.0
12,June 2022,15757.0
11,June 2021,16606.0


# Yearly Sales Performance In terms Of Revenue Generated

In [29]:
Yearly_Sales_M = Bakery.groupby('Year')['Total_Price'].sum().reset_index()

Yearly_Sales_M

,Year,Total_Price
0,2021,306088.77
1,2022,256627.93


# Year Over Year Sales Performance In Terms Of Revenue generated

In [30]:
Month_Yearly_Sales_M = Bakery.groupby('Month_Year')['Total_Price'].sum().reset_index()\
                                                                .sort_values(by = 'Month_Year',ascending = False)
Month_Yearly_Sales_M

,Month_Year,Total_Price
20,September 2022,21882.50
19,September 2021,22753.45
18,October 2021,22806.00
17,November 2021,18188.40
16,May 2022,29837.65
15,May 2021,29705.65
14,March 2022,19649.36
13,March 2021,20277.25
12,June 2022,25456.95
11,June 2021,24877.35


# Merge Dataframes

In [31]:
#Merge Yearly_Sales with Yearly_Sales_M
Yearly_Sales_Df = pd.merge(Yearly_Sales, Yearly_Sales_M, on = 'Year', how = 'inner')
Yearly_Sales_Df

,Year,Quantity,Total_Price
0,2021,204329.0,306088.77
1,2022,157827.0,256627.93


# Comment: 
Overall sales performance decreased by 16.16% 

# Why did the quantity ordered and sales reduce?

# Comment:
Check for the total unique number of orders 

In [32]:
Bakery_Cust = Bakery.groupby('Year')['Ticket_Number'].nunique().reset_index()
Bakery_Cust

,Year,Ticket_Number
0,2021,75758
1,2022,59927


In [33]:
#Add column percentage difference

In [34]:
Bakery_Cust['Percent_Change'] = Bakery_Cust['Ticket_Number'].pct_change()
Bakery_Cust

,Year,Ticket_Number,Percent_Change
0,2021,75758,NaN
1,2022,59927,-0.208968


# Comment:
The reason for the decline in the quantity ordered and hence sales, can be linked to the decline in number of orders. A 0.2% decline in the unique number of tickets was observed 

In [35]:
#Merge Month_Yearly_Sales with Month_Yearly_Sales_M
Month_Yearly_Sales_Df = pd.merge(Month_Yearly_Sales, Month_Yearly_Sales_M, on = 'Month_Year', how = 'inner')
Month_Yearly_Sales_Df

,Month_Year,Quantity,Total_Price
0,September 2022,13463.0,21882.50
1,September 2021,15296.0,22753.45
2,October 2021,15044.0,22806.00
3,November 2021,11868.0,18188.40
4,May 2022,17908.0,29837.65
5,May 2021,19540.0,29705.65
6,March 2022,11891.0,19649.36
7,March 2021,13895.0,20277.25
8,June 2022,15757.0,25456.95
9,June 2021,16606.0,24877.35


# Product Analysis

In [36]:
# Product performance in terms of quantity sold
Product_Sales = Bakery.groupby(['Hour_Type','Day_Type','Day_Of_Month','Date_of_Month','Month','Year','Article'])\
                                                ['Quantity'].sum().reset_index().sort_values(by = 'Quantity', ascending = False)
Product_Sales

,Hour_Type,Day_Type,Day_Of_Month,Date_of_Month,Month,Year,Article,Quantity
47030,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0
45964,Morning,Weekend,Sunday,7,August,2022,TRADITIONAL BAGUETTE,539.0
47532,Morning,Weekend,Sunday,17,July,2022,TRADITIONAL BAGUETTE,498.0
30395,Morning,Weekday,Monday,15,August,2022,TRADITIONAL BAGUETTE,472.0
49681,Morning,Weekend,Sunday,31,July,2022,TRADITIONAL BAGUETTE,456.0
...,...,...,...,...,...,...,...,...
37269,Morning,Weekday,Tuesday,18,January,2022,FINANCIER X5,1.0
18045,Mid-day,Weekday,Tuesday,12,April,2022,DIVERS VIENNOISERIE,1.0
37271,Morning,Weekday,Tuesday,18,January,2022,GAL FRANGIPANE 4P,1.0
37272,Morning,Weekday,Tuesday,18,January,2022,GAL POMME 6P,1.0


# Comment:
 The most ordered food from the restaurant is Traditional Baguette , and it receives the most order on sundays 

In [37]:
# Sales performance in terms of revenue generated
Product_Sales_M = Bakery.groupby(['Hour_Type','Day_Type','Day_Of_Month','Date_of_Month','Month','Year','Article'])\
                                        ['Total_Price'].sum().reset_index().sort_values(by = 'Total_Price', ascending = False)
Product_Sales_M

,Hour_Type,Day_Type,Day_Of_Month,Date_of_Month,Month,Year,Article,Total_Price
47030,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,734.50
45964,Morning,Weekend,Sunday,7,August,2022,TRADITIONAL BAGUETTE,700.70
47532,Morning,Weekend,Sunday,17,July,2022,TRADITIONAL BAGUETTE,647.40
30395,Morning,Weekday,Monday,15,August,2022,TRADITIONAL BAGUETTE,613.60
49681,Morning,Weekend,Sunday,31,July,2022,TRADITIONAL BAGUETTE,592.80
...,...,...,...,...,...,...,...,...
328,Evening,Weekday,Friday,5,March,2021,COUPE,0.15
13843,Mid-day,Weekday,Monday,7,March,2022,COUPE,0.15
15992,Mid-day,Weekday,Thursday,10,March,2022,COUPE,0.15
18206,Mid-day,Weekday,Tuesday,14,December,2021,COUPE,0.15


# Comment:

Traditional Baguette generated the most revenue in the bakery

In [38]:
# Merge DataFrames Product_Sales, Product_Sales_M
Product_Sales_Df = pd.merge(Product_Sales, Product_Sales_M, on = 'Article', how = 'inner')
Product_Sales_Df

,Hour_Type_x,Day_Type_x,Day_Of_Month_x,Date_of_Month_x,Month_x,Year_x,Article,Quantity,Hour_Type_y,Day_Type_y,Day_Of_Month_y,Date_of_Month_y,Month_y,Year_y,Total_Price
0,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,Morning,Weekend,Sunday,14,August,2022,734.5
1,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,Morning,Weekend,Sunday,7,August,2022,700.7
2,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,Morning,Weekend,Sunday,17,July,2022,647.4
3,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,Morning,Weekday,Monday,15,August,2022,613.6
4,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,Morning,Weekend,Sunday,31,July,2022,592.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45076944,Mid-day,Weekday,Tuesday,13,July,2021,PATES,1.0,Mid-day,Weekday,Tuesday,13,July,2021,4.5
45076945,Morning,Weekday,Monday,28,February,2022,PLAT 6.50E,1.0,Morning,Weekday,Monday,28,February,2022,6.5
45076946,Morning,Weekday,Tuesday,27,September,2022,SACHET DE VIENNOISERIE,1.0,Morning,Weekday,Tuesday,27,September,2022,4.0
45076947,Evening,Weekday,Wednesday,2,June,2021,CAKE,1.0,Evening,Weekday,Wednesday,2,June,2021,3.0


In [39]:
#Drop columns, Month_y and Year_y
Product_Sales_Df.drop(['Month_y', 'Year_y','Hour_Type_y','Day_Type_y','Day_Of_Month_y','Date_of_Month_y'], \
                      axis = 1, inplace = True)
Product_Sales_Df

,Hour_Type_x,Day_Type_x,Day_Of_Month_x,Date_of_Month_x,Month_x,Year_x,Article,Quantity,Total_Price
0,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,734.5
1,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,700.7
2,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,647.4
3,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,613.6
4,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,592.8
...,...,...,...,...,...,...,...,...,...
45076944,Mid-day,Weekday,Tuesday,13,July,2021,PATES,1.0,4.5
45076945,Morning,Weekday,Monday,28,February,2022,PLAT 6.50E,1.0,6.5
45076946,Morning,Weekday,Tuesday,27,September,2022,SACHET DE VIENNOISERIE,1.0,4.0
45076947,Evening,Weekday,Wednesday,2,June,2021,CAKE,1.0,3.0


In [40]:
#Rename Columns 'Month_x', Year_x
Product_Sales_Df.rename(columns = { 'Hour_Type_x': 'Hour_Type' , 'Day_Type_x' : 'Day_Type', 'Day_Of_Month_x' : 'Day_Of_Month', \
                                   'Month_x' : 'Month', 'Date_of_Month_x' : 'Date_of_Month', 'Year_x' : 'Year' }, inplace = True)
Product_Sales_Df

,Hour_Type,Day_Type,Day_Of_Month,Date_of_Month,Month,Year,Article,Quantity,Total_Price
0,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,734.5
1,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,700.7
2,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,647.4
3,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,613.6
4,Morning,Weekend,Sunday,14,August,2022,TRADITIONAL BAGUETTE,565.0,592.8
...,...,...,...,...,...,...,...,...,...
45076944,Mid-day,Weekday,Tuesday,13,July,2021,PATES,1.0,4.5
45076945,Morning,Weekday,Monday,28,February,2022,PLAT 6.50E,1.0,6.5
45076946,Morning,Weekday,Tuesday,27,September,2022,SACHET DE VIENNOISERIE,1.0,4.0
45076947,Evening,Weekday,Wednesday,2,June,2021,CAKE,1.0,3.0


# MARKET BASKET ANALYSIS

Market basket analysis using Apriori algorithm will be used to gain insight on pasteries commonly bought together from the bakery. This will enable the bakery to identify best food combinations for its cross-selling strategy to optimize sales.

# Groupby Ticket_Number and get unique number of products per Ticket_Number. 

In [41]:
N_Article_Df = Bakery.groupby('Ticket_Number')['Article'].nunique().reset_index()
N_Article_Df

,Ticket_Number,Article
0,150040.0,2
1,150041.0,2
2,150042.0,1
3,150043.0,2
4,150044.0,1
...,...,...
135680,288908.0,1
135681,288910.0,1
135682,288911.0,4
135683,288912.0,1


# Check how many unique Article combinations

In [42]:
N_Article_Df['Article'].unique()

array([ 2,  1,  3,  5,  4,  6,  9,  7, 10,  8, 11, 12], dtype=int64)

# Get Top 20 Best Sellers In The Bakery

In [43]:
Frequency_desc = Bakery.groupby('Article')['Quantity'].sum().reset_index().sort_values('Quantity', ascending = False)
Frequency_desc[:20]

,Article,Quantity
138,TRADITIONAL BAGUETTE,118093.0
30,CROISSANT,29822.0
83,PAIN AU CHOCOLAT,25367.0
29,COUPE,23601.0
5,BANETTE,22870.0
2,BAGUETTE,22156.0
23,CEREAL BAGUETTE,7478.0
124,SPECIAL BREAD,5488.0
55,FORMULE SANDWICH,5285.0
133,TARTELETTE,5051.0


# Get Least 20 Performing Products In The Bakery

In [58]:
Frequency_Asc = Bakery.groupby('Article')['Quantity'].sum().reset_index().sort_values('Quantity', ascending = True)
Frequency_Asc[:20]

,Article,Quantity
103,"PLATPREPARE6,00",1.0
20,CAKE,1.0
110,REDUCTION SUCREES 24,1.0
98,PLAT 6.50E,1.0
117,SACHET DE VIENNOISERIE,1.0
43,DOUCEUR D HIVER,1.0
141,TROIS CHOCOLAT,1.0
89,PAIN NOIR,1.0
65,GALETTE 8 PERS,2.0
18,BUCHE 8PERS,2.0


# Comment:
The quality of these foods should be re-evaluated and customer review on these foods should be obtained and analyzed to understand why these foods have low demand.

# Create customer's shopping  basket, data which contains quantity of each item bought per transaction( Ticket Number)

In [45]:
Basket_Df =  Bakery.groupby(['Ticket_Number', 'Article'])['Quantity'].sum().unstack().reset_index().fillna(0)\
                                                                                                .set_index('Ticket_Number')
Basket_Df

Article,12 MACARON,ARMORICAIN,BAGUETTE,BAGUETTE APERO,BAGUETTE GRAINE,BANETTE,BANETTINE,BOISSON 33CL,BOTTEREAU,BOULE 200G,...,THE,TRADITIONAL BAGUETTE,TRAITEUR,TRIANGLES,TROIS CHOCOLAT,TROPEZIENNE,TROPEZIENNE FRAMBOISE,TULIPE,VIENNOISE,VIK BREAD
Ticket_Number,,,,,,,,,,,,,,,,,,,,,
150040.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150041.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150042.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150043.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150044.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288908.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288910.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288911.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Encode the data, 0 for if item was not bought,and 1 for if item was bought. Reason being that in market basket analysis, we are 
concerned about whether an item was bought or not, and not necessarily the number of items bought per transaction.

In [47]:
def encode(x):
    if x <= 0:
        return False
    if x>= 1:
        return True
Encoded_Basket = Basket_Df.applymap(encode)
Encoded_Basket

Article,12 MACARON,ARMORICAIN,BAGUETTE,BAGUETTE APERO,BAGUETTE GRAINE,BANETTE,BANETTINE,BOISSON 33CL,BOTTEREAU,BOULE 200G,...,THE,TRADITIONAL BAGUETTE,TRAITEUR,TRIANGLES,TROIS CHOCOLAT,TROPEZIENNE,TROPEZIENNE FRAMBOISE,TULIPE,VIENNOISE,VIK BREAD
Ticket_Number,,,,,,,,,,,,,,,,,,,,,
150040.0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150041.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150042.0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
150043.0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150044.0,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288908.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
288910.0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
288911.0,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False


# Filter for basket with more than 1 item in it, so as to find associations between them for our market basket analysis

In [48]:
Filtered_Basket_Df = Encoded_Basket[(Encoded_Basket > 0).sum(axis = 1) > 2]
Filtered_Basket_Df

Article,12 MACARON,ARMORICAIN,BAGUETTE,BAGUETTE APERO,BAGUETTE GRAINE,BANETTE,BANETTINE,BOISSON 33CL,BOTTEREAU,BOULE 200G,...,THE,TRADITIONAL BAGUETTE,TRAITEUR,TRIANGLES,TROIS CHOCOLAT,TROPEZIENNE,TROPEZIENNE FRAMBOISE,TULIPE,VIENNOISE,VIK BREAD
Ticket_Number,,,,,,,,,,,,,,,,,,,,,
150046.0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
150052.0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150058.0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
150061.0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
150062.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288891.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
288897.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
288901.0,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


# We are interested in food purchased atleast 40 times 

The dataset has 135685 transations in total 

min_support=40/135685 = 0.03


In [49]:
Frequent_itemset = apriori(Filtered_Basket_Df, use_colnames = True, min_support = 0.03)
Frequent_itemset

,support,itemsets
0,0.133328,(BAGUETTE)
1,0.116071,(BANETTE)
2,0.061227,(BOULE 200G)
3,0.086364,(BOULE 400G)
4,0.035184,(BRIOCHE)
5,0.080966,(CAMPAGNE)
6,0.055278,(CEREAL BAGUETTE)
7,0.036445,(CHAUSSON AUX POMMES)
8,0.074071,(COMPLET)
9,0.041724,(COOKIE)


# Compute association rules with their corresponding support, confidence and lift, using lift metric

In [50]:
Rule = association_rules(Frequent_itemset, metric = 'lift')
Rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(COUPE),(BAGUETTE),0.500020,0.133328,0.058902,0.117800,0.883535,-0.007764,0.982398
1,(BAGUETTE),(COUPE),0.133328,0.500020,0.058902,0.441785,0.883535,-0.007764,0.895677
2,(CROISSANT),(BAGUETTE),0.262086,0.133328,0.040936,0.156194,1.171498,0.005993,1.027098
3,(BAGUETTE),(CROISSANT),0.133328,0.262086,0.040936,0.307033,1.171498,0.005993,1.064862
4,(PAIN AU CHOCOLAT),(BAGUETTE),0.244120,0.133328,0.036011,0.147515,1.106403,0.003463,1.016641
...,...,...,...,...,...,...,...,...,...
65,"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",(CROISSANT),0.125094,0.262086,0.089043,0.711811,2.715946,0.056258,2.560522
66,"(TRADITIONAL BAGUETTE, CROISSANT)",(PAIN AU CHOCOLAT),0.128679,0.244120,0.089043,0.691978,2.834586,0.057630,2.453981
67,(PAIN AU CHOCOLAT),"(TRADITIONAL BAGUETTE, CROISSANT)",0.244120,0.128679,0.089043,0.364751,2.834586,0.057630,1.371622
68,(CROISSANT),"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",0.262086,0.125094,0.089043,0.339747,2.715946,0.056258,1.325109


# Interpretation Of Result:

1. antecedent: Items which Customer already bought

2. consequent: Will customer buy this item?
    
3. antecedent support: frequency of antecedent in the entire bakery store transaction used for the market basket analysis
    
4. consequent support: frequency of consequent in the entire bakery store transaction used for the market basket analysis
    
5. support: frequency of both antecedent and consequent in the entire bakery store transactions used for the market basket analysis.
    
6. confidence: confidence that a customer will buy the consequent if antecedent is is bought by the customer
    
7. lift: relationship between the antecedent and the consequent
    
#Notes on lift: 
If lift value is greater than 1, it means the customer will likely buy the consequent given that said customer has bought the antecedent.
    
If lift value is less than 1, it means the customer is unlikely to buy the consequent given that said customer has bought the antecedent.

If lift value is equal to 1, it means there is no relationship between the antecedent and the consequent.


# Get Best Combo Deals

In [51]:
#Sort values by 'lift'
Rule.sort_values('lift', ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
67,(PAIN AU CHOCOLAT),"(TRADITIONAL BAGUETTE, CROISSANT)",0.244120,0.128679,0.089043,0.364751,2.834586,0.057630,1.371622
66,"(TRADITIONAL BAGUETTE, CROISSANT)",(PAIN AU CHOCOLAT),0.128679,0.244120,0.089043,0.691978,2.834586,0.057630,2.453981
65,"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",(CROISSANT),0.125094,0.262086,0.089043,0.711811,2.715946,0.056258,2.560522
68,(CROISSANT),"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",0.262086,0.125094,0.089043,0.339747,2.715946,0.056258,1.325109
29,(CROISSANT),(PAIN AU CHOCOLAT),0.262086,0.244120,0.172609,0.658599,2.697853,0.108629,2.214054
...,...,...,...,...,...,...,...,...,...
38,(TARTELETTE),(TRADITIONAL BAGUETTE),0.068555,0.511800,0.030692,0.447701,0.874758,-0.004394,0.883941
7,(BANETTE),(COUPE),0.116071,0.500020,0.049959,0.430414,0.860794,-0.008079,0.877796
6,(COUPE),(BANETTE),0.500020,0.116071,0.049959,0.099913,0.860794,-0.008079,0.982049
36,(SPECIAL BREAD),(TRADITIONAL BAGUETTE),0.085379,0.511800,0.035223,0.412552,0.806080,-0.008474,0.831052


In [52]:
# Get length of antecedent
Rule['antecedent_len'] = Rule['antecedents'].apply(lambda x: len(x))
Rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(COUPE),(BAGUETTE),0.500020,0.133328,0.058902,0.117800,0.883535,-0.007764,0.982398,1
1,(BAGUETTE),(COUPE),0.133328,0.500020,0.058902,0.441785,0.883535,-0.007764,0.895677,1
2,(CROISSANT),(BAGUETTE),0.262086,0.133328,0.040936,0.156194,1.171498,0.005993,1.027098,1
3,(BAGUETTE),(CROISSANT),0.133328,0.262086,0.040936,0.307033,1.171498,0.005993,1.064862,1
4,(PAIN AU CHOCOLAT),(BAGUETTE),0.244120,0.133328,0.036011,0.147515,1.106403,0.003463,1.016641,1
...,...,...,...,...,...,...,...,...,...,...
65,"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",(CROISSANT),0.125094,0.262086,0.089043,0.711811,2.715946,0.056258,2.560522,2
66,"(TRADITIONAL BAGUETTE, CROISSANT)",(PAIN AU CHOCOLAT),0.128679,0.244120,0.089043,0.691978,2.834586,0.057630,2.453981,2
67,(PAIN AU CHOCOLAT),"(TRADITIONAL BAGUETTE, CROISSANT)",0.244120,0.128679,0.089043,0.364751,2.834586,0.057630,1.371622,1
68,(CROISSANT),"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",0.262086,0.125094,0.089043,0.339747,2.715946,0.056258,1.325109,1


In [53]:
#Get Length of consequent
Rule['consequent_len'] = Rule['consequents'].apply(lambda x: len(x))
Rule

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
0,(COUPE),(BAGUETTE),0.500020,0.133328,0.058902,0.117800,0.883535,-0.007764,0.982398,1,1
1,(BAGUETTE),(COUPE),0.133328,0.500020,0.058902,0.441785,0.883535,-0.007764,0.895677,1,1
2,(CROISSANT),(BAGUETTE),0.262086,0.133328,0.040936,0.156194,1.171498,0.005993,1.027098,1,1
3,(BAGUETTE),(CROISSANT),0.133328,0.262086,0.040936,0.307033,1.171498,0.005993,1.064862,1,1
4,(PAIN AU CHOCOLAT),(BAGUETTE),0.244120,0.133328,0.036011,0.147515,1.106403,0.003463,1.016641,1,1
...,...,...,...,...,...,...,...,...,...,...,...
65,"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",(CROISSANT),0.125094,0.262086,0.089043,0.711811,2.715946,0.056258,2.560522,2,1
66,"(TRADITIONAL BAGUETTE, CROISSANT)",(PAIN AU CHOCOLAT),0.128679,0.244120,0.089043,0.691978,2.834586,0.057630,2.453981,2,1
67,(PAIN AU CHOCOLAT),"(TRADITIONAL BAGUETTE, CROISSANT)",0.244120,0.128679,0.089043,0.364751,2.834586,0.057630,1.371622,1,2
68,(CROISSANT),"(PAIN AU CHOCOLAT, TRADITIONAL BAGUETTE)",0.262086,0.125094,0.089043,0.339747,2.715946,0.056258,1.325109,1,2


# Filter for association rules with greater than 50% confidence and greater than 1.1 lift value to get best combo deals

In [55]:
Rule1 = Rule[(Rule['confidence'] > 0.5) & (Rule['lift'] > 1.1)] 
Rule1[:5]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
9,(BOULE 200G),(COUPE),0.061227,0.50002,0.059139,0.965894,1.931713,0.028524,14.659800,1,1
11,(BOULE 400G),(COUPE),0.086364,0.50002,0.082306,0.953011,1.905947,0.039122,10.640357,1,1
13,(CAMPAGNE),(COUPE),0.080966,0.50002,0.075726,0.935280,1.870486,0.035241,7.725260,1,1
14,(COMPLET),(COUPE),0.074071,0.50002,0.065088,0.878723,1.757378,0.028051,4.122645,1,1
19,(MOISSON),(COUPE),0.072889,0.50002,0.063197,0.867027,1.733986,0.026751,3.760014,1,1


# Comment: 

The table above displays the top 5 most frequently bought together foods from the bakery. 

There is a greater than 90% confidence that a customer who purchases Boule will also purchase coupe

There is a greater than 90% confidence that a customer who purchases Campagne will also purchase coupe.

There is a greater than 80% confidence that a customer who purchases complet will also purchase coupe

There is a greater than 80% confidence that a customer who purchases maisson will also purchase coupe

# Get Best Combo Deals For Atleast Two Antecedents And One Consequent 

In [56]:
Rule2 = Rule[(Rule['antecedent_len'] >= 2 ) & (Rule['confidence'] > 0.5) & (Rule['lift'] > 1.1)] 
Rule2[:3]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
43,"(BOULE 400G, TRADITIONAL BAGUETTE)",(COUPE),0.032032,0.50002,0.030219,0.943419,1.886765,0.014203,8.836608,2,1
47,"(COMPLET, TRADITIONAL BAGUETTE)",(COUPE),0.036957,0.50002,0.032938,0.891258,1.782446,0.014459,4.597858,2,1
54,"(MOISSON, TRADITIONAL BAGUETTE)",(COUPE),0.037114,0.50002,0.031362,0.845011,1.689955,0.012804,3.225900,2,1


# Comment:

There is a greater than 90% confidence that a customer who purchases both Traditional Baguette and Boule will also purchase Coupe

There is a greater than 85% confidence that a customer who purchases both Complet and Traditional Baguette will purchase Coupe

There is a greater than 80% chance that a customer who purchases both Traditional Baguette and Maisson will purchase Coupe



#  Get Best Combo Deals For The Most Sold Food In The Bakery-- 

This can be used to implement 'weekend specials'. Weekend specials because from analysis, our most sold food is sold mostly during the weekend

The best seller in the bakery is Traditional Baguette

In [63]:
Rule3 = Rule[(Rule['antecedents']== {'TRADITIONAL BAGUETTE'}) & (Rule['lift'] > 1.1)]
Rule3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
41,(TRADITIONAL BAGUETTE),(VIK BREAD),0.5118,0.089082,0.053741,0.105004,1.178727,0.008149,1.017789,1,1
63,(TRADITIONAL BAGUETTE),"(COUPE, VIK BREAD)",0.5118,0.076750,0.045940,0.089761,1.169524,0.006659,1.014294,1,2


In [64]:
Rule4 = Rule[(Rule['consequents']== {'TRADITIONAL BAGUETTE'}) & (Rule['lift'] > 1.1)]
Rule4

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len,consequent_len
40,(VIK BREAD),(TRADITIONAL BAGUETTE),0.089082,0.5118,0.053741,0.603273,1.178727,0.008149,1.230568,1,1
58,"(COUPE, VIK BREAD)",(TRADITIONAL BAGUETTE),0.076750,0.5118,0.045940,0.598563,1.169524,0.006659,1.216130,2,1


# Comment:

Rule 3 and Rule 4 shows the pasteries that are mostly bought together with Traditional Baguette from the bakery. These pasteries include:

Vik Bread and Coupe

